In [1]:
import tweepy
import emoji
import pandas as pd
from os import getenv

from transformers import pipeline
from helper import tweet_history, strip_all_entities, clean_hashtags, filter_chars, remove_mult_spaces


SENTIMENT_MODEL = "./sentiment_model"
SENTIMENT_TOKENIZER = "./sentiment_tokenizer"

EMOTION_MODEL = "./emotion_model"
EMOTION_TOKENIZER = "./emotion_tokenizer"


sentiment_pipe = pipeline(task="sentiment-analysis", model=SENTIMENT_MODEL, tokenizer=SENTIMENT_TOKENIZER)
emotion_pipe = pipeline(task="sentiment-analysis", model=EMOTION_MODEL, tokenizer=EMOTION_TOKENIZER)


auth = tweepy.OAuthHandler(getenv('TWITTER_API_KEY'), getenv('TWITTER_API_KEY_SECRET'))
auth.set_access_token(getenv('TWITTER_ACCESS_TOKEN'), getenv('TWITTER_ACCESS_TOKEN_SECRET'))
api = tweepy.API(auth, wait_on_rate_limit=True, parser=tweepy.parsers.JSONParser())

client = tweepy.Client(bearer_token=getenv('TWITTER_BEARER_TOKEN'), access_token=getenv('TWITTER_ACCESS_TOKEN')) 

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def clean_text(text):
    return remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(emoji.demojize(text)))))

In [ ]:
user_name = 'Tearing_Spaces'
user_id = api.get_user(screen_name=user_name)['id']
results = client.get_users_mentions(id=user_id, start_time='2023-09-05T00:00:00Z', end_time='2023-09-06T00:00:00Z', expansions=['author_id', 'referenced_tweets.id'], user_fields=['id', 'name', 'username'], tweet_fields=['created_at'], max_results=100)


tweets_data = [{'tweet_id': tweet.id, 'text': tweet.text, 'created_at': tweet.created_at} for tweet in results.data]

users_data = [{'user_id': user.id, 'name': user.name, 'username': user.username} for user in results.includes['users']]

combined_data = []
for tweet, user in zip(tweets_data, users_data):
    combined_data.append({**tweet, **user})

df = pd.DataFrame(combined_data)

df['text_cleaned'] = df['text'].apply(clean_text)
df['sentiment'] = df['text_cleaned'].apply(lambda x: sentiment_pipe(x))
df['emotion'] = df['text_cleaned'].apply(lambda x: emotion_pipe(x))
df

In [ ]:
space = '1OwxWwXEqWDxQ'
results = client.get_space(id=space, expansions=['creator_id', 'invited_user_ids', 'speaker_ids', 'host_ids', 'topic_ids'], user_fields=['created_at'], space_fields=['participant_count', 'started_at', 'title', 'invited_user_ids'])

results_dict = results._asdict()

print(f'Participants: {results_dict["data"]["participant_count"]}')

users_data = results_dict.get('includes', {}).get('users', [])
users_list = [{'id': user.id, 'name': user.name, 'username': user.username} for user in users_data]
df = pd.DataFrame(users_list)
df
